<a href="https://colab.research.google.com/github/rjnakano/manejo-inventarios-pdg/blob/main/Notebooks/clusterboot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install --upgrade kmodes

Requirement already up-to-date: kmodes in /usr/local/lib/python3.7/dist-packages (0.11.0)


In [1]:
import pathlib
import pandas as pd
import numpy as np
from pylab import *

from sklearn.preprocessing import StandardScaler

from kmodes.kprototypes import KPrototypes

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()

from plotnine import *
import plotnine

In [3]:
plot_kwds = {'alpha' : 0.5, 's' : 1, 'linewidths':0}

In [104]:
if str(pathlib.Path().absolute()).find('content') >= 0:
    from google.colab import drive
    drive.mount('/content/gdrive')
    sourcepath = '/content/gdrive/MyDrive/TDGdata'
else:
    sourcepath = '../Datos/Transformed'
print(sourcepath) 

../Datos/Transformed


In [116]:
# Dataset de inventarios sin dimensiones de los productos
data1 = pd.read_csv("{}/dfNoSupervisado_fixed.csv".format(sourcepath))

In [117]:
data1['AISLE']=data1['AISLE'].astype(str)
data1.drop('Unnamed: 0', inplace=True, axis=1)

In [118]:
data1.dtypes

AISLE        object
IG           object
HTS          object
PLANNING     object
ABC          object
UOM          object
CAT          object
COSTX       float64
QTYVAR      float64
dtype: object

In [108]:
data1.head(5)

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR
0,17,IG56,HTS119,1/1,B,PC,CAT17,-0.029976,-0.037185
1,17,IG56,HTS119,1/1,B,PC,CAT17,0.092472,-0.006928
2,17,IG56,HTS119,1/1,C,PC,CAT17,0.266130,0.020450
3,17,IG56,HTS119,1/1,B,PC,CAT17,-0.025112,-0.044259
4,15,IG56,HTS119,1/1,C,PC,CAT17,-0.077982,-0.046471


In [109]:
# scikit-learn bootstrap
from sklearn.utils import resample

In [110]:
data1.shape[0]

16239

In [13]:
def add2set (nuevos):
  conjunto = {999999}
  conjunto.update(nuevos)
  conjunto.discard(999999)
  return(conjunto)


In [114]:
data1.head()

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR,Cluster
0,17,IG56,HTS119,1/1,B,PC,CAT17,-0.029976,-0.037185,1
1,17,IG56,HTS119,1/1,B,PC,CAT17,0.092472,-0.006928,1
2,17,IG56,HTS119,1/1,C,PC,CAT17,0.266130,0.020450,1
3,17,IG56,HTS119,1/1,B,PC,CAT17,-0.025112,-0.044259,1
4,15,IG56,HTS119,1/1,C,PC,CAT17,-0.077982,-0.046471,1


In [163]:
def get_jaccard(df, nBootstrap, k, gamma = 0.5 ):
    jaccard = [ [ 0 for i in range(k) ] for j in range(nBootstrap) ]
    catColumnsPos = [df.columns.get_loc(col) for col in list(df.select_dtypes('object').columns)]
    cluster_idx = unique(df.Cluster)
    
    orig_set = get_orig(df, cluster_idx)
        
    df.drop('Cluster', inplace = True, axis=1)
    for boot_iter in range(nBootstrap):
        print('bootstrap # ',boot_iter);
        
        boot = get_boot(df,k,gamma,boot_iter);
        
        boot_set = {}
        for set_iter in cluster_idx:
            boot_set[set_iter] = add2set(boot[boot['Cluster']==set_iter].index)
        
        for set_iter in cluster_idx:
            for jaccard_iter in cluster_idx:
                if jaccard_iter == 0:
                    jaccard[boot_iter][set_iter] = 0
                jaccard[boot_iter][set_iter] = max(jaccard[boot_iter][set_iter] , calculate_jaccard(orig_set[set_iter] , boot_set[jaccard_iter]))
            print('jaccard = {}'.format(jaccard[boot_iter][set_iter]))
    return jaccard

def calculate_jaccard(orig,boot):
    return len(orig & boot) / len(orig | boot) 
    
def get_boot(df,k,gamma,i):
    boot = resample(df, replace=True, n_samples=df.shape[0], random_state=i)
    # Fit the cluster
    kprototypebs = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 1)
    kprototypebs.fit_predict(boot, categorical = catColumnsPos)
    boot['Cluster'] = kprototypebs.labels_
    print("--- %s seconds ---" % (time.time() - start_time));
    return boot
    
def get_orig(df, cluster_idx):
    orig_set = {}
    for main_iter in cluster_idx:
        orig_set[main_iter] = add2set(df[df['Cluster']==main_iter].index)
    return orig_set

In [164]:
# Inicializamos la matriz de distancias con ceros
import warnings
warnings.filterwarnings('ignore')
import time
start_time = time.time()
test = {}
nBootstrap = 100
gamma = 0.5
kValues = [2,3]
for k in kValues:
    print('Generando K = {}'.format(k))
    kprototype = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 0)
    kprototype.fit_predict(data1, categorical = catColumnsPos)
    data1['Cluster'] = kprototype.labels_
    print("--- %s seconds ---" % (time.time() - start_time))

    test[k] = get_jaccard(data1, nBootstrap, k, gamma)
    
    
    

Generando K = 2
--- 23.84663987159729 seconds ---
bootstrap #  0
--- 41.37248396873474 seconds ---
jaccard = 0.6092843326885881
jaccard = 0.6328904710444345
bootstrap #  1
--- 55.51541209220886 seconds ---
jaccard = 0.586073500967118
jaccard = 0.6288600838734274
bootstrap #  2
--- 71.11126589775085 seconds ---
jaccard = 0.6150870406189555
jaccard = 0.6322138276934798
bootstrap #  3
--- 85.25499248504639 seconds ---
jaccard = 0.5783365570599613
jaccard = 0.6347229283172343
bootstrap #  4
--- 103.3775942325592 seconds ---
jaccard = 0.6312741312741312
jaccard = 0.6301361149980919
bootstrap #  5
--- 118.82784056663513 seconds ---
jaccard = 0.6335877862595419
jaccard = 0.6320106829454407
bootstrap #  6
--- 137.18660521507263 seconds ---
jaccard = 0.5996131528046421
jaccard = 0.630809333079026
bootstrap #  7
--- 151.54949975013733 seconds ---
jaccard = 0.5846702317290553
jaccard = 0.6335707925200356
bootstrap #  8
--- 167.6415364742279 seconds ---
jaccard = 0.6365384615384615
jaccard = 0.633

In [166]:

for k in kValues:
    jaccard = pd.DataFrame(test[k])
    print(jaccard.describe())
    jaccard.to_csv('{}/jaccard-fixed-{}'.format(sourcepath,k))

                0           1
count  100.000000  100.000000
mean     0.560679    0.615779
std      0.164570    0.062457
min      0.030806    0.316258
25%      0.584454    0.628662
50%      0.612715    0.630234
75%      0.634429    0.632387
max      0.680851    0.638896
                0           1           2
count  100.000000  100.000000  100.000000
mean     0.036750    0.399169    0.168284
std      0.139988    0.128968    0.146946
min      0.000397    0.305600    0.045110
25%      0.000829    0.314485    0.077899
50%      0.001199    0.322861    0.081155
75%      0.018453    0.600049    0.319032
max      0.875000    0.613727    0.541076


In [24]:
# np.savetxt("jaccard.csv", jaccard, delimiter=",")

In [27]:
# np.savetxt("/content/gdrive/MyDrive/TDGdata/jaccard.csv", jaccard, delimiter=",")

,0,1,2
count,4.000000,4.000000,4.000000
mean,0.508756,0.511189,0.628641
std,0.141342,0.141364,0.028642
min,0.382716,0.385293,0.605825
25%,0.388210,0.390662,0.611650
50%,0.510297,0.509807,0.619417
75%,0.630843,0.630333,0.636408
max,0.631715,0.639850,0.669903


In [24]:
boot.index

Int64Index([  338, 14512,    74,  7696, 10982,   866,  5876,   153,  8940,
             4026,
            ...
             7563,    72,  5567, 14758,  6438,  3012,  8186,  7117, 12450,
             4552],
           dtype='int64', length=16239)